In [1]:
import os, math
import warnings

from MDAnalysis import Universe, Writer
import nglview as nv
import mdtraj

from moleidoscope.linker import Linker, view_linkers
from moleidoscope.mirror import Mirror
from moleidoscope.line import Line
from moleidoscope.geo.quaternion import Quaternion

warnings.filterwarnings('ignore') # MDAnalysis creates a lot of warnings

/home/kutay/anaconda3/lib/python3.5/site-packages/MDAnalysis/__init__.py:190: UserWarning: #####
MDAnalysis on python 3 is highly experimental!
It is mostly non functional and dramatically untested.
Use at your own risks!!!

  ''')
/home/kutay/anaconda3/lib/python3.5/site-packages/jupyter_core/application.py:106: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  help="""Generate default config file."""
/home/kutay/anaconda3/lib/python3.5/site-packages/notebook/nbextensions.py:286: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  overwrite = Bool(False, config=True, help="Force overwrite of existing files")
/home/kutay/anaconda3/lib/python3.5/site-packages/notebook/services/config/manager.py:15: DeprecationWarning: metadata {'config': True} was

## Animation using trajectories ##
### Creating triangular prism ###

In [2]:
# Create wedge (trigonal prism) geometry with three planes
# defined by nine total points
p1 = [-3**0.5/3, -1, -1]
p2 = [-3**0.5/3, -1, 1]
p3 = [2*3**0.5/3, 0, 1]
mir1 = Mirror(p1, p2, p3, size=7)

p1 = [-3**0.5/3, 1, -1]
p2 = [-3**0.5/3, 1, 1]
p3 = [2*3**0.5/3, 0, 1]
mir2 = Mirror(p1, p2, p3, size=7)

p1 = [-3**0.5/3, -1, -1]
p2 = [-3**0.5/3, -1, 1]
p3 = [-3**0.5/3, 1, 1]
mir3 = Mirror(p1, p2, p3, size=7)

gs = 7 # Grid size for plane visualization

# Convert mirror planes to linkers
plane1 = mir1.to_linker(grid_size=gs)
plane2 = mir2.to_linker(grid_size=gs)
plane3 = mir3.to_linker(grid_size=gs)

# Add xyz axis
lxyz = Line.xyz()

# Join linkers
prism = plane1.join(plane2, plane3)

# Join all and visualize
view_linkers(prism, lxyz)

### Creating the animation ###

In [3]:
# Create a linker and rotate it multiple times
link = Linker(123)
link.center(coor=[0, 0, 0])

rpt = 5
frames = 0
all_linkers = link.join(plane1)
link2 = link.reflect(mir1)
all_linkers = plane1.join(link2)

all_linkers = all_linkers.join(plane2)
link3 = link.reflect(mir2)
all_linkers = all_linkers.join(link3)

all_linkers = all_linkers.join(plane3)
link4 = link.reflect(mir3)
all_linkers = all_linkers.join(link4)


# Determine rotation -> [angle, p1, p2] where p1----p2 is the rotation axis
rotation_info = [math.pi/48, [0, 0, 0], [0, 0, 1]]
rotation_info2 = [math.pi/48, [0, 0, 0], [0, 0, 1]]
y_axis = Line([0, -7, 0], [0, 6, 0]).to_linker()            # Create y axis line

linker_paths = []
# Rotate linker and export
for i in range(200):
    all_linkers = all_linkers.rotate(rotation_info)
    link = link.rotate(rotation_info2)
    linker1 = all_linkers.join(link)
    linker1.name = 'prism-%i' % i
    linker_paths.append(linker1.export())
    
# Create a trajectory from file path list and save as XTC
T = mdtraj.load(linker_paths)
traj_path = os.path.join('linkers', 'rot_traj.xtc')
T.save_xtc(traj_path, force_overwrite=True)

# Create a MDAnalysis Universe and show animation
u = Universe(linker_paths[0], traj_path)
nv.show_mdanalysis(u)

In [ ]:
def repeat_scene(linker, count, name='animation', start=0):
    linkers = []
    for i in range(count):
        linker.name = '%s-%i' % (name, i + start)
        linkers.append(linker.export())
    return linkers

In [ ]:
# Create a linker and rotate it multiple times
link = Linker(123)
link.center(coor=[0, 0, 0])

rpt = 5
frames = 0
# Add first linker
linker_paths = repeat_scene(link, 5)
frames += rpt

# Add plane 1
all_linkers = link.join(plane1)
linker_paths += repeat_scene(all_linkers, rpt, start=frames)
frames += rpt

# Reflect linker
link2 = link.reflect(mir1)
all_linkers = all_linkers.join(link2)
linker_paths += repeat_scene(all_linkers, rpt, start=frames)
frames += rpt

# Add plane 2
all_linkers = all_linkers.join(plane2)
linker_paths += repeat_scene(all_linkers, rpt, start=frames)
frames += rpt

# Reflect linker
link3 = link.reflect(mir2)
all_linkers = all_linkers.join(link3)
linker_paths += repeat_scene(all_linkers, rpt, start=frames)
frames += rpt

# Add plane 3
all_linkers = all_linkers.join(plane3)
linker_paths += repeat_scene(all_linkers, rpt, start=frames)
frames += rpt

# Reflect linker
link4 = link.reflect(mir3)
all_linkers = all_linkers.join(link4)
linker_paths += repeat_scene(all_linkers, rpt)
frames += rpt

# Determine rotation -> [angle, p1, p2] where p1----p2 is the rotation axis
rotation_info = [math.pi/12, [0, 0, 0], [0, 1, 0]]
y_axis = Line([0, -7, 0], [0, 6, 0]).to_linker()            # Create y axis line

# linker_paths = []
# # Rotate linker and export
# for i in range(240):
#     link = link.rotate(rotation_info)
#     linker1 = link.join(y_axis)
#     linker1.name = 'rotation-%i' % i
#     linker_paths.append(linker1.export())


In [ ]:
# Create a trajectory from file path list and save as XTC
T = mdtraj.load(linker_paths)
traj_path = os.path.join('linkers', 'rot_traj.xtc')
T.save_xtc(traj_path, force_overwrite=True)

# Create a MDAnalysis Universe and show animation
u = Universe(linker_paths[0], traj_path)
nv.show_mdanalysis(u)

In [ ]:
for i in range(3,5):
    print(i)